# Tema 3 - Metoda Simplex

In [ ]:
import numpy as np
import math

In [ ]:
def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]

In [ ]:
def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])

In [ ]:
def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)
        
    if (all([r == math.inf for r in restrictions])):
        raise Exception("Linear program is unbounded.")

    row = restrictions.index(min(restrictions))
    return row, column

In [ ]:
def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
   
    return new_tableau

In [ ]:
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns[:-1]:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
        
    return solutions

In [ ]:
def simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

In [ ]:
def to_objective_function_value(c, solution):
    return sum(np.array(c) * np.array(solution))

In [ ]:
c = [2, 3, 0, 0, 0] # functia pe care dorim sa o maximizam 2*x1 + 3*x2
A = [
    [4, 8, 1, 0, 0],   # 4*x1 + 8*x2 + x3 <= 12
    [2, 1, 0, 1, 0],   # 2*x1 + x2 + x4 <= 2
    [3, 2, 0, 0, 1]    # 3*x1 + 2*x2 + x5 <= 4
] 
b = [12, 3, 4]

# afisam valorile x1, x2, x3, x4, x5
solution = simplex(c, A, b)
print('solution: ', solution)

# afisam rezultatul functiei pe care am maximizat-o
result = to_objective_function_value(c, simplex(c, A, b))
print('Rezultatul este: ', result)

solution:  [0.5, 1.25, 0, 0.75, 0]
Primal:  4.75


## Tema 4 - Metoda Duala

In [ ]:
def can_be_improved_for_dual(tableau):
    rhs_entries = [row[-1] for row in tableau[:-1]]
    return any([entry < 0 for entry in rhs_entries])

def get_pivot_position_for_dual(tableau):
    rhs_entries = [row[-1] for row in tableau[:-1]]
    min_rhs_value = min(rhs_entries)
    row = rhs_entries.index(min_rhs_value)
    
    columns = []
    for index, element in enumerate(tableau[row][:-1]):
        if element < 0:
            columns.append(index)
    columns_values = [tableau[row][c] / tableau[-1][c] for c in columns]
    column_min_index = columns_values.index(min(columns_values))
    column = columns[column_min_index]

    return row, column

def dual_simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved_for_dual(tableau):
        pivot_position = get_pivot_position_for_dual(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

In [ ]:
c = [12, 3, 4, 0, 0] # functia pe care dorim sa o minimizam 12*y1 + 3*y2 + 4*y3

A = [
    [-4, -2, -3,  1,  0],   # -4*y1 -2*y2 -3*y3 + y4 >= -2
    [-8, -1, -2,  0,  1]    # -8*y1 -*y2 -2*y3 + y5 >= -3
]
b = [-2, -3]

# afisam valorile y1, y2, y3, y4, y5
solution = dual_simplex(c, A, b)
print('solution: ', solution)

result = to_objective_function_value(c, dual_simplex(c, A, b))
print('Rezultatul este: ', result)

solution:  [0.3125, 0, 0.25, 0, 0]
Rezultatul este:  4.75


In [ ]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('Tema3+4.ipynb')

--2021-04-19 12:29:03--  https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1864 (1.8K) [text/plain]
Saving to: ‘colab_pdf.py’

colab_pdf.py        100%[===================>]   1.82K  --.-KB/s    in 0s      

2021-04-19 12:29:03 (32.4 MB/s) - ‘colab_pdf.py’ saved [1864/1864]

Mounted at /content/drive/




Extracting templates from packages: 100%
[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/Tema3+4.ipynb to pdf
[NbConvertApp] Writing 36533 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'File ready to be Downloaded and Saved to Drive'